In [ ]:
# WB
import requests
import pandas as pd
import time
from datetime import datetime, timedelta
from requests.exceptions import ChunkedEncodingError, RequestException
from tqdm import tqdm


tokens = {
    "ХТС Вижн": "",
    "ХТС Рус": " "
}

final_file_path = ""


final_data = pd.read_excel(final_file_path)
last_week = final_data[final_data["МП"] == "WB"]["Неделя"].max()
if pd.isna(last_week):
    last_week = 0
print(f"Последняя неделя WB в финальном файле: {last_week}")

url = "https://seller-analytics-api.wildberries.ru/api/v2/nm-report/detail"


def get_page(page_number, start_date, end_date, headers, retries=3):
    payload = {
        "page": page_number,
        "period": {
            "begin": start_date,
            "end": end_date
        },
        "limit": 500
    }
    for attempt in range(retries):
        try:
            response = requests.post(url, headers=headers, json=payload, timeout=60)
            response.raise_for_status()
            return response.json().get("data", {}).get("cards", [])
        except (ChunkedEncodingError, RequestException) as e:
            print(f"Попытка {attempt + 1}/{retries} — ошибка: {e}. Повтор через 60 сек.")
            time.sleep(60)
    return []


def collect_data(start_date, end_date, headers):
    all_data = []
    page = 1
    while True:
        data = get_page(page, start_date, end_date, headers)
        if not data:
            break
        all_data.extend(data)
        if len(data) < 500:
            break
        page += 1
        time.sleep(1)
    return all_data


today = datetime.today()
prev_week = today.isocalendar()[1] - 1
prev_week_year = today.isocalendar()[0]
monday_last_week = datetime.strptime(f'{prev_week_year}-W{prev_week:02}-1', "%G-W%V-%u")
date_list = [monday_last_week + timedelta(days=i) for i in range(7)]


data_frames = []

for company, token in tokens.items():
    print(f"\n Обработка для: {company}")
    headers = {
        "Content-Type": "application/json",
        "Authorization": token
    }

    for single_date in tqdm(date_list, desc=f"Загрузка {company}"):
        begin = single_date.replace(hour=0, minute=0, second=0).strftime("%Y-%m-%d %H:%M:%S")
        end = single_date.replace(hour=23, minute=59, second=59).strftime("%Y-%m-%d %H:%M:%S")

        data = collect_data(begin, end, headers)
        if data:
            df = pd.json_normalize(data)
            df.rename(columns={
                "statistics.selectedPeriod.begin": "Дата начала",
                "statistics.selectedPeriod.end": "Дата конца",
                "statistics.selectedPeriod.ordersCount": "Количество заказов",
                "statistics.selectedPeriod.ordersSumRub": "Сумма заказов",
                "statistics.selectedPeriod.buyoutsCount": "Количество выкупов",
                "statistics.selectedPeriod.buyoutsSumRub": "Сумма выкупов",
                "stocks.stocksMp": "Остатки MP",
                "stocks.stocksWb": "Остатки Wb",
                "nmID": "ID",
                "vendorCode": "Артикул",
                "brandName": "Бренд",
                "object.name": "Название"
            }, inplace=True)

            df = df[[
                "Дата начала", "Дата конца", "Количество заказов", "Сумма заказов",
                "Количество выкупов", "Сумма выкупов", "Остатки MP", "Остатки Wb",
                "ID", "Артикул", "Бренд", "Название"
            ]]
            df["Юридическое лицо"] = company
            df["Дата продажи"] = pd.to_datetime(df["Дата начала"])
            df["Неделя"] = df["Дата продажи"].apply(lambda x: x.isocalendar()[1])
            df["МП"] = "WB"
            df = df[df["Количество заказов"] > 0]
            data_frames.append(df)


if data_frames:
    full_df = pd.concat(data_frames, ignore_index=True)
    full_df = full_df[full_df["Неделя"] > last_week]

    final_block = full_df.rename(columns={
        'Сумма выкупов': 'Продано, руб.',
        'Количество выкупов': 'Продано, шт.'
    })[['Юридическое лицо', 'МП', 'Неделя', 'Артикул', 'Бренд', 'Продано, руб.', 'Продано, шт.', 'Дата продажи']]

    final_data = pd.concat([final_data, final_block], ignore_index=True)
    final_data.drop_duplicates(inplace=True)

    try:
        with pd.ExcelWriter(final_file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            final_data.to_excel(writer, index=False)
        print(f"\n Данные WB успешно добавлены в файл: {final_file_path}")
    except Exception as e:
        print(f" Ошибка при сохранении: {e}")
else:
    print("Нет новых данных для добавления.")



Последняя неделя WB в финальном файле: 13

🔄 Обработка для: ХТС Вижн


Загрузка ХТС Вижн:   0%|          | 0/7 [00:00<?, ?it/s]

Попытка 1/3 — ошибка: 429 Client Error: Too Many Requests for url: https://seller-analytics-api.wildberries.ru/api/v2/nm-report/detail. Повтор через 60 сек.


<ipython-input-2-c845db881ebb>:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Юридическое лицо"] = company
<ipython-input-2-c845db881ebb>:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Дата продажи"] = pd.to_datetime(df["Дата начала"])
<ipython-input-2-c845db881ebb>:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

Попытка 1/3 — ошибка: 429 Client Error: Too Many Requests for url: https://seller-analytics-api.wildberries.ru/api/v2/nm-report/detail. Повтор через 60 сек.


...

Попытка 1/3 — ошибка: 429 Client Error: Too Many Requests for url: https://seller-analytics-api.wildberries.ru/api/v2/nm-report/detail. Повтор через 60 сек.


Загрузка ХТС Рус:  57%|█████▋    | 4/7 [01:11<01:19, 26.54s/it]<ipython-input-2-c845db881ebb>:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Юридическое лицо"] = company
<ipython-input-2-c845db881ebb>:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Дата продажи"] = pd.to_datetime(df["Дата начала"])
<ipython-input-2-c845db881ebb>:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

Попытка 1/3 — ошибка: 429 Client Error: Too Many Requests for url: https://seller-analytics-api.wildberries.ru/api/v2/nm-report/detail. Повтор через 60 сек.


Загрузка ХТС Рус: 100%|██████████| 7/7 [02:35<00:00, 22.21s/it]



 Данные WB успешно добавлены в файл: /content/Продажи_финальный файл 2025.xlsx


In [ ]:
import pandas as pd
import requests
import time
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

final_file_path = "/content/Продажи_финальный файл 2025.xlsx"
reference_file_path = "/content/Справочник_Ozon.xlsx"

entities = {
    "ХТС Вижн": {
        "api_key": "",
        "businessId":
    },
    "ХТС Рус": {
        "api_key": "",
        "businessId":
    }
}

today = datetime.today()
prev_week = today.isocalendar()[1] - 1
prev_year = today.isocalendar()[0]
monday = datetime.strptime(f"{prev_year}-W{prev_week}-1", "%G-W%V-%u")
sunday = monday + timedelta(days=6)
date_from_str = monday.strftime("%Y-%m-%d")
date_to_str = sunday.strftime("%Y-%m-%d")

print(f"Загружаем данные за период: {date_from_str} — {date_to_str}")


final_data = pd.read_excel(final_file_path)
last_week = final_data[final_data['МП'] == 'YM']['Неделя'].max()
if pd.isna(last_week):
    last_week = 0

reference_df = pd.read_excel(reference_file_path)
reference_df.rename(columns={'Артикул': 'Ваш SKU', 'Бренд': 'Бренд_HTS'}, inplace=True)


def generate_and_download_report(api_key, business_id):
    url_generate = "https://api.partner.market.yandex.ru/reports/shows-sales/generate"
    headers = {"Content-Type": "application/json", "Api-Key": api_key}
    body = {
        "businessId": business_id,
        "dateFrom": date_from_str,
        "dateTo": date_to_str,
        "grouping": "OFFERS"
    }

    response = requests.post(url_generate, json=body, headers=headers)
    if response.status_code != 200:
        print(f"Ошибка создания отчета: {response.status_code}")
        return None

    report_id = response.json()["result"]["reportId"]
    url_info = f"https://api.partner.market.yandex.ru/reports/info/{report_id}"

    for attempt in range(10):
        resp = requests.get(url_info, headers=headers)
        if resp.status_code == 200 and resp.json()["result"]["status"] == "DONE":
            return resp.json()["result"]["file"]
        elif resp.json()["result"]["status"] == "ERROR":
            print("Ошибка генерации отчета")
            return None
        time.sleep(10)
    print("Таймаут ожидания готовности отчета.")
    return None

def process_report(file_url, legal_entity):
    response = requests.get(file_url, stream=True)
    if response.status_code != 200:
        print(f"шибка при скачивании отчета: {response.status_code}")
        return pd.DataFrame()

    df = pd.read_excel(file_url)
    df["Юридическое лицо"] = legal_entity
    df["МП"] = "YM"
    df["Неделя"] = monday.isocalendar()[1]
    df.rename(columns={"День": "Дата продажи"}, inplace=True)


    df = df.merge(reference_df[['Ваш SKU', 'Бренд_HTS']], on='Ваш SKU', how='left')

    df = df.rename(columns={
        'Ваш SKU': 'Артикул',
        'Бренд_HTS': 'Бренд',
        'Доставлено за период на сумму, ₽': 'Продано, руб.',
        'Доставлено за период, шт.': 'Продано, шт.'
    })[['Юридическое лицо', 'МП', 'Неделя', 'Артикул', 'Бренд', 'Продано, руб.', 'Продано, шт.', 'Дата продажи']]

    df['Дата продажи'] = pd.to_datetime(df['Дата продажи'], errors='coerce').dt.strftime('%d.%m.%Y %H:%M:%S')
    df = df.dropna(subset=['Дата продажи'])

    return df

all_data = pd.DataFrame()
for entity_name, creds in entities.items():
    print(f"\n Обработка: {entity_name}")
    file_url = generate_and_download_report(creds['api_key'], creds['businessId'])
    if file_url:
        df = process_report(file_url, entity_name)
        all_data = pd.concat([all_data, df], ignore_index=True)


if not all_data.empty:
    all_data = all_data.loc[:, ~all_data.columns.duplicated()]
    all_data = all_data[all_data["Неделя"] > last_week]

    missing_cols = [col for col in final_data.columns if col not in all_data.columns]
    for col in missing_cols:
        all_data[col] = None

    all_data = all_data[final_data.columns]  # Переставим порядок столбцов
    final_data = pd.concat([final_data, all_data], ignore_index=True)
    final_data.drop_duplicates(inplace=True)

    try:
        with pd.ExcelWriter(final_file_path, engine="openpyxl", mode='a', if_sheet_exists='replace') as writer:
            final_data.to_excel(writer, index=False)
        print(f"\n Данные успешно добавлены в файл: {final_file_path}")
    except Exception as e:
        print(f"Ошибка при сохранении: {e}")
else:
    print("Нет новых данных для добавления.")


Загружаем данные за период: 2025-03-31 — 2025-04-06

📥 Обработка: ХТС Вижн


<ipython-input-7-1ce21deccd73>:98: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Дата продажи'] = pd.to_datetime(df['Дата продажи'], errors='coerce').dt.strftime('%d.%m.%Y %H:%M:%S')



📥 Обработка: ХТС Рус


<ipython-input-7-1ce21deccd73>:98: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Дата продажи'] = pd.to_datetime(df['Дата продажи'], errors='coerce').dt.strftime('%d.%m.%Y %H:%M:%S')
<ipython-input-7-1ce21deccd73>:123: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_data = pd.concat([final_data, all_data], ignore_index=True)
